In [8]:
from prep_data import Data_prepare
import numpy as np 
import os 
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
# import prep_data

###  Data preprocessing 
1. covert mat data to numpy array 
2. combine male and female data to one array 
3. if needed, remove certain rows 
4. get postive and negative of Xs 
    - order difference or net vs neg 
5. reshape each X: 128*8*2 = 2048 
    - input: data 
6. generate labels for x 
    - input: data and label_type (0:zeros, 1: ones )
7. combine net and neg 
    - input: netX,netY,negX,negY

In [9]:
def netural_vs_negative_helper(data,position):
    arr = []
    index = 0
    for i in range(data.shape[0]):
#         print(data[i].shape)
        temp = []
        for j in position:
            for k in data[i][j[0]][j[1]]:
                temp.append(k)
        arr.append(np.array(temp))
    return np.array(arr)

In [10]:
dp = Data_prepare() 
# pos = [[0,3],[0,2],[0,1],[0,0]]
# neg = [[1,3],[1,2],[1,1],[1,0]]
# netural vs postive 
pos = [[0,0],[0,1],[1,2],[1,3],[2,0],[2,1],[2,2],[2,3]]
neg = [[0,2],[0,3],[1,0],[1,1],[3,0],[3,1],[3,2],[3,3]] 
m_file = "./raw_data/Emotrans1_Boy_data_raw.mat" 
f_file = "./raw_data/Emotrans1_girl_data_raw_update.mat"
# index = [11,18,36]
df = dp.combine_male_female(m_file,f_file)
# df = dp.remove_person(df,index)
pos_df = dp.netural_vs_negative_helper(df,pos)
neg_df = dp.netural_vs_negative_helper(df,neg)
pos_sequeezed = dp.squeeze_feature_size(pos_df)
neg_sequeezed = dp.squeeze_feature_size(neg_df)
pos_labels = dp.generate_labels(pos_sequeezed,1)
neg_labels = dp.generate_labels(neg_sequeezed,0)
x,y = dp.combine_net_neg(pos_sequeezed,pos_labels,neg_sequeezed,neg_labels)

In [11]:
x[0].shape

(192, 2048)

In [12]:
model = RandomForestClassifier(n_estimators=150) 
# model = Pipeline([
#             ('clf', LogisticRegression(solver='saga', 
#                                 penalty='l1',
#                                 C=alpha,
#                                 max_iter=1000))
#             ])

In [13]:
def get_x_train_test(data,index):
    x_test = data[index]
    x_train = np.delete(data,index,axis=0)
    return x_train,x_test.reshape(1,-1)

# get y train and test set for witin CV 
def get_y_train_test(data,index):
    y_test = data[index]
    y_train = np.delete(data,index,axis=0)
    return y_train.astype(int).ravel(),y_test.astype(int)
def model_train(x_train,x_test,y_train,y_test,model):
    model.fit(x_train,y_train.ravel())
    y_pred = model.predict(x_test)
    return accuracy_score(y_test,y_pred)

def within_train(x,y,model):
    # with_in
    all_accuracy = []
    avg_accuracy = []
    # train a model 
    # return y_test, y_pred 
    for d in range(x.shape[0]):
        # training set and test set
        accuracy =[]
        print("within # person:",d)
        for i in range(x[d].shape[0]):
            x_train,x_test = get_x_train_test(x[d],i)
            y_train,y_test = get_y_train_test(y[d],i)
            acc= model_train(x_train,x_test,y_train,y_test,model)
            accuracy.append(acc)
        temp_acc = sum(accuracy)/len(accuracy)
        avg_accuracy.append(temp_acc)
        print(temp_acc)
    return avg_accuracy


In [ ]:
accs = within_train(x,y,model)

within # person: 0
0.7135416666666666
within # person: 1
0.6875
within # person: 2
0.6354166666666666
within # person: 3
0.6145833333333334
within # person: 4
0.6875
within # person: 5
0.625
within # person: 6
0.6927083333333334
within # person: 7
0.578125
within # person: 8
